In [1]:
""" Este script extrae información de los campos del HTML 
    del cvlac a partir de una base de datos inicial de los perfiles
"""
# Importar librerias/Modulos
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
import time

In [2]:
#parser = pag_principal()

def links_pag_principal():

    link = 'https://www.argentina.gob.ar/ciencia/financiamiento'
    encabezados = {
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
        }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text    

    response = scrapy.Selector(text=resp)
    links = response.xpath('//div[@class="col-xs-12 col-sm-6 col-md-6"]/a/@href').getall()

    urls = []
    url = 'https://www.argentina.gob.ar'
    for link in links:
        urls.append(url+link)


    return urls


def links_proyectos(link):
    #link = 'https://www.argentina.gob.ar/ciencia/financiamiento'

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    response = scrapy.Selector(text=resp)
    links = response.xpath('//nav/ul/li[@class=""]/a/@href').getall()

    urls = []
    url = 'https://www.argentina.gob.ar'
    for link in links:
        urls.append(url+link)


    return urls

def parser_proyecto(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    parser = html.fromstring(resp)

    return parser


def titulo(parser):
    titulo_proyecto = parser.xpath('//h1[@class="activities-sidbar"]//text()')
    return titulo_proyecto


def descripcion(parser):

    descripción = parser.xpath('//div[@class=""]//text()')

    text = ''
    for i in descripción:
        text = text+i
    
    return text

def fecha_cierre(parser):
    try:
        fecha_cierre= parser.xpath('//div[@class="media-body"]//text()')
        if fecha_cierre[0] != 'Fecha de cierre':
            fecha_cierre = 'Ventanilla permanente'
        else:
            fecha_cierre = fecha_cierre[1]

    except IndexError:
        fecha_cierre=''

    return fecha_cierre


def pdf(parser):

    ############# 1
    pdfs = parser.xpath('//div[@class="container"]//p//a/@href')
    
    urls=''

    for p in pdfs:
        if p.endswith('.pdf'):
            urls = urls + p + ', '
        else:
            urls = ''
    ############## 2
    if urls == '':
        pdfs = parser.xpath('//div[@class="downloads panel-pane pane-entity-field pane-node-field-download"]//a/@href')
        for p in pdfs:
            if p.endswith('.pdf'):
                urls = urls + p + ', '
            else:
                urls = ''
        
    ############## 3
    if urls == '':
        pdfs = parser.xpath('//div[@class="container"]//div[@class="panel-pane pane-atajos"]//a/@href')
        #print(pdfs)

        for p in pdfs:
            
            if p.startswith('https://www.argentina.gob.ar') == False: 
                        
                print(p)
                try:
                    parser_subproyecto = parser_proyecto('https://www.argentina.gob.ar' + p)
                except:
                    continue

                pdfs2 = parser_subproyecto.xpath('//div[@class="container"]//a/@href')
                    #//article[@class="content_format col-md-8 col-md-push-4"]
                for pdf in pdfs2:
                    if pdf.endswith('.pdf'):
                        urls = urls + pdf + ', '
                    
                if urls == '':
                    pdfs2 = parser_subproyecto.xpath('//div[@class="downloads panel-pane pane-entity-field pane-node-field-download"]//div[@class="row row-flex"]//a/@href')
                    for pdf in pdfs2:
                        if pdf.endswith('.pdf'):
                            urls = urls + pdf + ', '
                        else:
                            urls = ''
                
        if urls == '':
            pdfs = parser.xpath('//div[@class="panel-pane pane-texto"]//a/@href')

            for p in pdfs:

                if p.startswith('https://www.argentina.gob'):
                    try:
                        
                        parser_subproyecto = parser_proyecto(p)

                    except:
                        continue

                    pdfs2 = parser_subproyecto.xpath('//p//a/@href')


                    urls = ''
                   
                    for f in pdfs2:

                        if f.endswith('.pdf') == True:
                            urls = urls + f + ', '


    return urls.strip(', ')

#####################################33

def links_cerradas_segundo_nivel(link):

    encabezados = {
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
        }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    response = scrapy.Selector(text=resp)
    links = response.xpath('//ul/li[@class="active"]//li/a/@href').getall()

    url = 'https://www.argentina.gob.ar'
    urls = []
    for link in links:
        urls.append(url + link)

    return urls

def descripcion_vertical(parser):

    try:
        links = parser.xpath('//div/p/a/@href')[-1]

        parser = parser_proyecto(links)
        contenidos = parser.xpath('//div[@class="field-item even"]//p/text()')

        contenido = ''
        for content in contenidos:
            contenido = contenido+content
    except:
        contenido = ''


    return contenido

def pdf_vertical(parser):
    try:
        links = parser.xpath('//div/p/a/@href')[-1]

        parser2 = parser_proyecto(links)
        links = parser2.xpath('//div[@class="downloads panel-pane pane-entity-field pane-node-field-download"]//a/@href')
        if links == '':
            links = parser2.xpath('//section/h3[1]//a/@href')
        urls = ''
        for url in links:
            urls = urls + url + ', '
    except:
        links = parser.xpath('//div[@class="downloads panel-pane pane-entity-field pane-node-field-download"]//a/@href')
        if links == '':
            links = parser.xpath('//section/h3[1]//a/@href')
        urls = ''
        for url in links:
            urls = urls + url + ', '

    
    return urls.rstrip(', ')
    



In [3]:
def argentina():
    # abiertas
    urls = links_pag_principal()

    links_abiertas = links_proyectos(urls[0])
    #parser_cerradas = parser(urls[1])
    argentina = pd.DataFrame()
    titulos = []
    descripciones = []
    fechas_cierre = []
    pdfs = []


    for link in links_abiertas:
    
        parser = parser_proyecto(link)
        
        titulos.append(titulo(parser)[0])
        descripciones.append(descripcion(parser) + '---' + descripcion_vertical(parser))
        fechas_cierre.append(fecha_cierre(parser))

        if pdf(parser) == '':
            pdfs.append(pdf_vertical(parser))
        else:
            pdfs.append(pdf(parser))
        

    argentina['Tútulo'] = titulos
    argentina['Descripción'] = descripciones
    argentina['Fecha Cierre'] = fechas_cierre
    argentina['Links pdf'] = pdfs
    argentina['Ano'] = 2021
    argentina['Estado de la convocatoria'] = 'Convocatoria Abierta'
    argentina['Link'] = links_abiertas

################# Cerradas

    links_cerradas = links_proyectos(urls[1])
    for link_cerrada in links_cerradas:
        
        links_segundo_nivel = links_cerradas_segundo_nivel(str(link_cerrada))
        cerradas = pd.DataFrame()
        
        titulos = []
        descripciones = []
        fechas_cierre = []
        pdfs = []

        for segundo_nivel in links_segundo_nivel:
            parser = parser_proyecto(segundo_nivel)

            titulos.append(titulo(parser)[0])
            descripciones.append(descripcion(parser) + '---' + descripcion_vertical(parser))
            fechas_cierre.append(fecha_cierre(parser))

            if pdf(parser) == '':
                pdfs.append(pdf_vertical(parser))
            else:
                pdfs.append(pdf(parser))

        year = link_cerrada.split('-')[-1]
        print(year)
        cerradas['Tútulo'] = titulos
        cerradas['Descripción'] = descripciones
        cerradas['Fecha Cierre'] = fechas_cierre
        cerradas['Links pdf'] = pdfs
        cerradas['Ano'] = year
        cerradas['Estado de la convocatoria'] = 'Convocatoria Cerrada'
        cerradas['Link'] = links_segundo_nivel

        argentina = argentina.append(cerradas)

    argentina.reset_index(drop=True, inplace=True)

    return argentina

In [4]:
argentina = argentina()

argentina.to_excel('Argentina.xlsx')

tificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/duban/anaconda3/envs/scraping-clacso/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.argentina.gob.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/duban/anaconda3/envs/scraping-clacso/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tenaris.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/duban/anaconda3/envs/scraping-clacso/lib/python3.8/site-packages/urllib3/connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host '

In [7]:
argentina['Link'].tolist()[7]

'https://www.argentina.gob.ar/ciencia/financiamiento/convocatorias-mincyt/sistemas-nacionales'

In [8]:
argentina.to_excel('Argentina.xlsx')